## JONS Paper (8.3)

In [ ]:
import Pkg; Pkg.activate(".")

In [ ]:
using Revise

In [ ]:
using ArgCheck
using CSV
using DataFrames
using Distributions
using HMMBase
using ParsimoniousMonitoring
using PyPlot
using Random

In [ ]:
using POMDPs
using POMDPModelTools
using POMDPSimulators
using DiscreteValueIteration

In [ ]:
include("thesis.jl");

### 8.3 Two paths between two anchors on RIPE Atlas

0: US
1: DE
4: HK
8: NP (nepal)
13: SG (singapore)
15: LV (latvia)
19: KZ (kazakhstan)
24: NZ
29: CZ

In [ ]:
plot_rtt(rtt; ax = gca(), kwargs...) = ax.plot(coalesce.(rtt, NaN); kwargs...)
read_rtt(source) = CSV.read(source, header = [:rtt], missingstring = "2.000000000000000000e+03");

In [ ]:
hk_kz = read_rtt("../data/jons_pair/rtt4_19.csv").rtt
hk_lv_kz = read_rtt("../data/jons_pair/rtt4_15_19.csv").rtt;

In [ ]:
fig, ax = subplots(figsize = (10, 3))
plot_rtt(hk_kz, ax = ax, label = "HK → KZ")
plot_rtt(hk_lv_kz, ax = ax, label = "HK → LV → KZ")
ax.set(xlabel = "Time", ylabel = "RTT (ms)")
ax.legend(loc = "upper left");

#### Models (Table 7)

In [ ]:
p1 = HMM([0.995 0.005; 0.005 0.995], [Normal(320, sqrt(30)), Normal(370, sqrt(30))])
p2 = HMM([0.995 0.005; 0.05 0.95], [Normal(350, sqrt(30)), Normal(440, sqrt(100))]);

In [ ]:
fig, ax = subplots(figsize = (10, 3))

Random.seed!(2020)
plot_rtt(rand(p1, 5000), ax = ax, label = "HMM: HK → KZ")
plot_rtt(rand(p2, 5000), ax = ax, label = "HMM: HK → LV → KZ")

ax.set(xlabel = "Time", ylabel = "RTT (ms)")
ax.legend(loc = "upper left");

#### MDP

In [ ]:
mdp = MonitoringMDP([p1, p2], [150, 150], [4, 4]);
smdp = SparseTabularMDP(mdp);

In [ ]:
solver = SparseValueIterationSolver(max_iterations=5000, belres=1e-6);
policy_mdp_90 = solve_sparse(solver, mdp, smdp, 0.9);

#### Benchmark

In [ ]:
# TODO: Initial state?

In [ ]:
# Set pdf(::Distribution, ::Missing) = 1.0
# (Equivalent of doing a prediction in the HMM filter)
ParsimoniousMonitoring.enablemissing(1.0);

In [ ]:
# In the JONS paper, time series are subsampled by a factor 5.  
# Not sure why, but we do the same here to keep the same model parameters.
data = hcat(hk_kz[1:5:end], hk_lv_kz[1:5:end]);

In [ ]:
logbook = benchmark(mdp, policy_mdp_90, data, (DiscreteBelief(150, 2), DiscreteBelief(150, 2)));

In [ ]:
predictor_p1 = map(logbook) do history
    state = history.s[1]
    (mdp.models[1].A^(state.timesteps+1))[state.laststate,1]
end
predictor_p2 = map(logbook) do history
    state = history.s[2]
    (mdp.models[2].A^(state.timesteps+1))[state.laststate,1]
end;

In [ ]:
instants_p1 = findall(map(h -> h.a[1], logbook))
instants_p2 = findall(map(h -> h.a[2], logbook));

In [ ]:
fig, axs = subplots(figsize = (10, 9), nrows = 3, sharex = true)
axs[1].plot(coalesce.(data[:,1], NaN), label = "HK → KZ")
axs[1].plot(coalesce.(data[:,2], NaN), label = "HK → LV → KZ")
axs[1].set(ylabel = "RTT (ms)")
axs[1].legend(loc = "upper left", ncol = 2)

axs[2].plot(predictor_p1)
axs[2].plot(predictor_p2)
axs[2].scatter(instants_p1, ones(length(instants_p1)) * 1.2, s = 5.0, label = "Mesures HK → KZ")
axs[2].scatter(instants_p2, ones(length(instants_p2)) * 1.1, s = 5.0, label = "Mesures HK → LV → KZ")
axs[2].set(ylabel = L"\gamma_{t-1,t}(i)", ylim = (0, 1.3))
axs[2].legend(loc = "lower right")

axs[3].plot(map(h -> h.path, logbook), drawstyle = "steps-mid")
axs[3].set(xlabel = "Temps", ylabel = "C(t)", ylim = (0.8, 2.2), yticks = [1, 2])

save_thesis("monitoring_atlas_2p_mdp_aio", clean = true, hwr = 0.4);